In [1]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop
from keras.models import model_from_json
%matplotlib inline 
# from keras.datasets import mnist
tf.logging.set_verbosity(tf.logging.ERROR)
# tf.set_random_seed(2017)
np.random.seed(2017)

Using TensorFlow backend.


# Load MNIST Data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# test data
test_images = mnist.test.images.reshape(10000, 28, 28, 1)
test_labels = mnist.test.labels[:]

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Data Agumentation
- image rotation
- image width shift

In [3]:
augmentation_size = 440000
images = np.concatenate((mnist.train.images.reshape(55000, 28, 28, 1), mnist.validation.images.reshape(5000, 28, 28, 1)), axis=0)
labels = np.concatenate((mnist.train.labels, mnist.validation.labels), axis=0)

datagen_list = [
                ImageDataGenerator(rotation_range=10),
                ImageDataGenerator(rotation_range=20),
                ImageDataGenerator(rotation_range=30),
                ImageDataGenerator(width_shift_range=0.1),
                ImageDataGenerator(width_shift_range=0.2),
                ImageDataGenerator(width_shift_range=0.3),
               ]

for datagen in datagen_list:
    datagen.fit(images)
    for image, label in datagen.flow(images, labels, batch_size=augmentation_size, shuffle=True, seed=2017):
        images = np.concatenate((images, image), axis=0)
        labels = np.concatenate((labels, label), axis=0)
        break

print(images.shape)
print(labels.shape)

(1800000, 28, 28, 1)
(1800000, 10)


# Train Parameter

In [4]:
epochs = 20

# Model 1 Architecture
1. Convolution + Convolution + MaxPool + Dropout
2. Convolution + Convolution + MaxPool + Dropout
3. Convolution + MaxPool + Dropout
4. Dense + Dropout
5. Dense + Dropout
6. Output

## Model 1 Parameter

In [5]:
model1_filter_size = 3
model1_conv1_filter_num_1 = 64
model1_conv1_filter_num_2 = 128
model1_conv1_filter_num_3 = 128
model1_conv1_filter_num_4 = 128
model1_conv1_filter_num_5 = 128
model1_keep_prob = 0.5
model1_pool_size = 2
model1_acitivation_fn = 'elu'
model1_output_acitivation_fn = 'softmax'
model1_padding = 'same'
mode1l_batch_size = 256
model1_optimizer = Adam(lr=0.0001)

In [5]:
model1 = Sequential([Convolution2D(filters=model1_conv1_filter_num_1, kernel_size=(model1_filter_size, model1_filter_size), padding='same', activation='elu', input_shape=(28, 28, 1)), 
                     Convolution2D(filters=model1_conv1_filter_num_2, kernel_size=(model1_filter_size, model1_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model1_pool_size),
                     Dropout(model1_keep_prob),
                     Convolution2D(filters=model1_conv1_filter_num_3, kernel_size=(model1_filter_size, model1_filter_size), padding='same', activation='elu'),
                     Convolution2D(filters=model1_conv1_filter_num_4, kernel_size=(model1_filter_size, model1_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model1_pool_size),
                     Dropout(model1_keep_prob),
                     Convolution2D(filters=model1_conv1_filter_num_5, kernel_size=(model1_filter_size, model1_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model1_pool_size),
                     Dropout(model1_keep_prob),
                     Flatten(),
                     Dense(1024, activation='elu'),
                     Dropout(model1_keep_prob),
                     Dense(1024, activation='elu'),
                     Dropout(model1_keep_prob),
                     Dense(10, activation=model1_output_acitivation_fn),
                     ])
model1.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(images, labels, batch_size=256, epochs=epochs, shuffle=True, verbose=1, validation_data=(test_images, test_labels))

Train on 1800000 samples, validate on 10000 samples
Epoch 1/20
1800000/1800000 [==============================] - 425s - loss: 0.2764 - acc: 0.9133 - val_loss: 0.0293 - val_acc: 0.9899
Epoch 2/20
1800000/1800000 [==============================] - 424s - loss: 0.1080 - acc: 0.9665 - val_loss: 0.0230 - val_acc: 0.9926
Epoch 3/20
1800000/1800000 [==============================] - 434s - loss: 0.0770 - acc: 0.9760 - val_loss: 0.0168 - val_acc: 0.9949
Epoch 4/20
1800000/1800000 [==============================] - 433s - loss: 0.0622 - acc: 0.9805 - val_loss: 0.0160 - val_acc: 0.9954
Epoch 5/20
1800000/1800000 [==============================] - 424s - loss: 0.0524 - acc: 0.9835 - val_loss: 0.0148 - val_acc: 0.9960
Epoch 6/20
1800000/1800000 [==============================] - 424s - loss: 0.0463 - acc: 0.9853 - val_loss: 0.0131 - val_acc: 0.9962
Epoch 7/20
1800000/1800000 [==============================] - 424s - loss: 0.0416 - acc: 0.9869 - val_loss: 0.0145 - val_acc: 0.9962
Epoch 8/20
180000

In [6]:
model_json = model1.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
model1.save_weights("model1.h5")
print("Saved model to disk")

Saved model to disk


# Model 2 Architecture
1. Convolution * 2 + MaxPool + Dropout
2. Convolution * 2 + MaxPool + Dropout
3. Convolution + MaxPool + Dropout
4. Dense + Dropout
5. Dense + Dropout
6. Output

## Model 2 Parameter

In [6]:
model2_filter_size = 5
model2_conv1_filter_num_1 = 64
model2_conv1_filter_num_2 = 128
model2_conv1_filter_num_3 = 128
model2_conv1_filter_num_4 = 128
model2_conv1_filter_num_5 = 128
model2_keep_prob = 0.5
model2_pool_size = (2, 2)
model2_acitivation_fn = 'elu'
model2_output_acitivation_fn = 'softmax'
model2_padding = 'same'
mode1l_batch_size = 256
model2_optimizer = Adam(lr=0.0001)

In [7]:
model2 = Sequential([Convolution2D(filters=model2_conv1_filter_num_1, kernel_size=(model2_filter_size, model2_filter_size), padding='same', activation='elu', input_shape=(28, 28, 1)), 
                     Convolution2D(filters=model2_conv1_filter_num_2, kernel_size=(model2_filter_size, model2_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model2_pool_size),
                     Dropout(model2_keep_prob),
                     Convolution2D(filters=model2_conv1_filter_num_3, kernel_size=(model2_filter_size, model2_filter_size), padding='same', activation='elu'),
                     Convolution2D(filters=model2_conv1_filter_num_4, kernel_size=(model2_filter_size, model2_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model2_pool_size),
                     Dropout(model2_keep_prob),
                     Convolution2D(filters=model2_conv1_filter_num_5, kernel_size=(model2_filter_size, model2_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model2_pool_size),
                     Dropout(model2_keep_prob),
                     Flatten(),
                     Dense(1024, activation='elu'),
                     Dropout(model2_keep_prob),
                     Dense(1024, activation='elu'),
                     Dropout(model2_keep_prob),
                     Dense(10, activation=model2_output_acitivation_fn),
                     ])
model2.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(images, labels, batch_size=256, epochs=epochs, shuffle=True, verbose=1, validation_data=(test_images, test_labels))

Train on 1800000 samples, validate on 10000 samples
Epoch 1/20
1800000/1800000 [==============================] - 607s - loss: 0.2144 - acc: 0.9333 - val_loss: 0.0209 - val_acc: 0.9936
Epoch 2/20
1800000/1800000 [==============================] - 606s - loss: 0.0838 - acc: 0.9742 - val_loss: 0.0170 - val_acc: 0.9947
Epoch 3/20
1800000/1800000 [==============================] - 610s - loss: 0.0570 - acc: 0.9823 - val_loss: 0.0137 - val_acc: 0.9958
Epoch 4/20
1800000/1800000 [==============================] - 613s - loss: 0.0442 - acc: 0.9862 - val_loss: 0.0146 - val_acc: 0.9960
Epoch 5/20
1800000/1800000 [==============================] - 612s - loss: 0.0360 - acc: 0.9887 - val_loss: 0.0128 - val_acc: 0.9964
Epoch 6/20
1800000/1800000 [==============================] - 613s - loss: 0.0313 - acc: 0.9903 - val_loss: 0.0139 - val_acc: 0.9960
Epoch 7/20
1800000/1800000 [==============================] - 612s - loss: 0.0276 - acc: 0.9912 - val_loss: 0.0108 - val_acc: 0.9964
Epoch 8/20
180000

In [8]:
model_json = model2.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model_json)
model2.save_weights("model2.h5")
print("Saved model to disk")

Saved model to disk


# Model 3 Architecture
1. Convolution + Convolution + MaxPool + Dropout
2. Convolution + Convolution + MaxPool + Dropout
3. Convolution + MaxPool + Dropout
4. Dense + Dropout
5. Dense + Dropout
6. Output

## Model 3 Parameter

In [7]:
model3_filter_size = 7
model3_conv1_filter_num_1 = 64
model3_conv1_filter_num_2 = 128
model3_conv1_filter_num_3 = 128
model3_conv1_filter_num_4 = 128
model3_conv1_filter_num_5 = 128
model3_keep_prob = 0.5
model3_pool_size = (2, 2)
model3_acitivation_fn = 'elu'
model3_output_acitivation_fn = 'softmax'
model3_padding = 'same'
mode1l_batch_size = 256
model3_optimizer = Adam(lr=0.0001)

In [9]:
model3 = Sequential([Convolution2D(filters=model3_conv1_filter_num_1, kernel_size=(model3_filter_size, model3_filter_size), padding='same', activation='elu', input_shape=(28, 28, 1)), 
                     Convolution2D(filters=model3_conv1_filter_num_2, kernel_size=(model3_filter_size, model3_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model3_pool_size),
                     Dropout(model3_keep_prob),
                     Convolution2D(filters=model3_conv1_filter_num_3, kernel_size=(model3_filter_size, model3_filter_size), padding='same', activation='elu'),
                     Convolution2D(filters=model3_conv1_filter_num_4, kernel_size=(model3_filter_size, model3_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model3_pool_size),
                     Dropout(model3_keep_prob),
                     Convolution2D(filters=model3_conv1_filter_num_5, kernel_size=(model3_filter_size, model3_filter_size), padding='same', activation='elu'),
                     MaxPooling2D(pool_size=model3_pool_size),
                     Dropout(model3_keep_prob),
                     Flatten(),
                     Dense(1024, activation='elu'),
                     Dropout(model3_keep_prob),
                     Dense(1024, activation='elu'),
                     Dropout(model3_keep_prob),
                     Dense(10, activation=model3_output_acitivation_fn),
                     ])
model3.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model3.fit(images, labels, batch_size=256, epochs=epochs, shuffle=True, verbose=1, validation_data=(test_images, test_labels))

Train on 1800000 samples, validate on 10000 samples
Epoch 1/20
1800000/1800000 [==============================] - 842s - loss: 0.1917 - acc: 0.9408 - val_loss: 0.0209 - val_acc: 0.9936
Epoch 2/20
1800000/1800000 [==============================] - 840s - loss: 0.0708 - acc: 0.9785 - val_loss: 0.0146 - val_acc: 0.9956
Epoch 3/20
1800000/1800000 [==============================] - 840s - loss: 0.0462 - acc: 0.9858 - val_loss: 0.0117 - val_acc: 0.9967
Epoch 4/20
1800000/1800000 [==============================] - 839s - loss: 0.0341 - acc: 0.9894 - val_loss: 0.0139 - val_acc: 0.9967
Epoch 5/20
1800000/1800000 [==============================] - 839s - loss: 0.0276 - acc: 0.9914 - val_loss: 0.0139 - val_acc: 0.9966
Epoch 6/20
1800000/1800000 [==============================] - 838s - loss: 0.0234 - acc: 0.9927 - val_loss: 0.0145 - val_acc: 0.9964
Epoch 7/20
1800000/1800000 [==============================] - 840s - loss: 0.0205 - acc: 0.9936 - val_loss: 0.0127 - val_acc: 0.9972
Epoch 8/20
180000

In [10]:
model_json = model3.to_json()
with open("model3.json", "w") as json_file:
    json_file.write(model_json)
model3.save_weights("model3.h5")
print("Saved model to disk")

Saved model to disk


# Evaluate

In [8]:
# load json and create model
def model_open(name, test_images, test_labels):
    json_file = open(name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name + '.h5')
    print("Loaded model from disk")
    loaded_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])
    prob = loaded_model.predict_proba(test_images)
    acc = np.mean(np.equal(np.argmax(prob, axis=1), np.argmax(test_labels, axis=1)))
    print('\nmodel : %s, test accuracy : %4f\n' %(name, acc))
    return prob

In [9]:
prob_1 = model_open('model1', test_images, test_labels)
prob_2 = model_open('model2', test_images, test_labels)
prob_3 = model_open('model3', test_images, test_labels)

Loaded model from disk
10000/10000 [==============================] - 1s     

model : model1, test accuracy : 0.996900

Loaded model from disk
 9984/10000 [============================>.] - ETA: 0s
model : model2, test accuracy : 0.997200

Loaded model from disk
 9984/10000 [============================>.] - ETA: 0s
model : model3, test accuracy : 0.997200



In [20]:
final_prob = (prob_1 * 1 + prob_2 * 2  + prob_3 * 3) / 6
final_score = np.mean(np.equal(np.argmax(final_prob, axis=1), np.argmax(test_labels, axis=1)))
print('final test accuracy : %4f' % final_score)

final test accuracy : 0.998000
